In [1]:
import bz2
import numpy as np
import pickle
import _pickle as cPickle
from tqdm.notebook import tqdm

In [2]:
event_file = "e0010GP_20201118/2020-11-18_e0010GP_Events.pbz2"
data_event = bz2.BZ2File(event_file,"rb")
data_event = cPickle.load(data_event)
chapter_segments = []
durations = []
# drop the first value of data_event['signal'] because it is the start of the recording
data_event['signal'] = data_event['signal'][1:]
for i in range(len(data_event['signal'])):
    if data_event['signal'][i][0] == 9:
        start_time = data_event['signal'][i][1]
    elif data_event['signal'][i][0] == 18:
        end_time = data_event['signal'][i][1]
        chapter_segments.append((start_time, end_time))
        durations.append(end_time - start_time)

In [3]:
chapter_segments = chapter_segments[53:]
durations = durations[53:]

In [4]:
import glob
# import every file that ends with .wav in the folder e0010GP_20201118/xmen
audio_files = glob.glob("e0010GP_20201118/xmen/*.wav")
audio_files.sort()
print(audio_files) 
# calculate the duration of each audio file, and map the name and the duration of the audio file
import wave
audio_duration = []
for audio_file in audio_files:
    with wave.open(audio_file, 'rb') as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        audio_duration.append(duration)
audio_duration = np.array(audio_duration)

['e0010GP_20201118/xmen\\xmen_01.wav', 'e0010GP_20201118/xmen\\xmen_02.wav', 'e0010GP_20201118/xmen\\xmen_03.wav', 'e0010GP_20201118/xmen\\xmen_04.wav', 'e0010GP_20201118/xmen\\xmen_05.wav', 'e0010GP_20201118/xmen\\xmen_06.wav', 'e0010GP_20201118/xmen\\xmen_07.wav', 'e0010GP_20201118/xmen\\xmen_08.wav', 'e0010GP_20201118/xmen\\xmen_09.wav', 'e0010GP_20201118/xmen\\xmen_10.wav', 'e0010GP_20201118/xmen\\xmen_11.wav', 'e0010GP_20201118/xmen\\xmen_12.wav', 'e0010GP_20201118/xmen\\xmen_13.wav', 'e0010GP_20201118/xmen\\xmen_14.wav', 'e0010GP_20201118/xmen\\xmen_15.wav', 'e0010GP_20201118/xmen\\xmen_16.wav', 'e0010GP_20201118/xmen\\xmen_17.wav', 'e0010GP_20201118/xmen\\xmen_18.wav', 'e0010GP_20201118/xmen\\xmen_19.wav', 'e0010GP_20201118/xmen\\xmen_20.wav', 'e0010GP_20201118/xmen\\xmen_21.wav', 'e0010GP_20201118/xmen\\xmen_22.wav', 'e0010GP_20201118/xmen\\xmen_23.wav', 'e0010GP_20201118/xmen\\xmen_24.wav']


In [5]:
# for each chapter, look for the audio file that has the closest duration to the chapter duration
chapter_audio = []
for i in range(len(durations)):
    chapter_duration = durations[i]
    diff = np.abs(audio_duration - chapter_duration/1024)
    index = np.argmin(diff)
    chapter_audio.append((chapter_segments[i], audio_files[index]))

In [6]:
idx_audio = np.hstack([np.arange(1, 9), np.arange(10, 16), np.arange(18, 21)])
print(audio_duration[idx_audio])
idx_seeg = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 19, 20, 21]
print(np.array(durations)[idx_seeg]/1024)

[244.288      178.00533333 333.71733333 316.032      468.672
 179.66933333 244.992      345.62133333 300.69333333 145.6
 190.93333333 163.02933333 260.032      143.68       178.624
 173.71733333 277.84533333]
[244.60742188 178.33984375 333.99707031 316.42480469 469.02050781
 179.9765625  245.22460938 345.95996094 300.9921875  145.94726562
 191.38574219 163.3828125  260.32128906 144.05273438 178.88671875
 174.08691406 278.1796875 ]


In [7]:
seeg_files = glob.glob("e0010GP_20201118/*.pbz2")
seeg_files.sort()
print(seeg_files)
contact_names = []
for seeg_file in seeg_files:
    # seeg_file is f"e0010GP_20201118/2020-11-18_e0010GP_{contact}.pbz2"
    contact = seeg_file.split("_")[-1].split(".")[0]
    contact_names.append(contact)

['e0010GP_20201118\\2020-11-18_e0010GP_A-AMY1.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY10.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY11.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY12.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY2.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY3.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY4.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY5.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY6.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY7.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY8.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_A-AMY9.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP1.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP10.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP11.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP12.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP2.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP3.pbz2', 'e0010GP_20201118\\2020-11-18_e0010GP_B-PHIP4.pbz2'

In [8]:
data_contacts = []
for contact in tqdm(contact_names):
    if contact == "Events":
        continue
    #Load data
    filename = f"e0010GP_20201118/2020-11-18_e0010GP_{contact}.pbz2"
    data =bz2.BZ2File(filename,"rb")
    data =cPickle.load(data)['signal']
    # Extract data according to chapter segments and concatenate
    data_contact = []
    for segment in chapter_segments:
        data_contact.append(data[segment[0]:segment[1]])
    # data_contact = np.concatenate(data_contact)
    data_contacts.append(data_contact)
# data_contacts = np.array(data_contacts)

  0%|          | 0/111 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
np.save("e0010GP_20201118_seeg_contacts.npy", data_contacts)
